# R Implementation of ADMM

The R package file is available on Canvas.

In [2]:
library(admmlasso)

In [4]:
lasso_obj <- function(X, y, beta, lambda) {
    (1/2)*(1/length(y))*norm(y - X%*%beta,"F")^2 + lambda*sum(abs(beta))
}

In [3]:
set.seed(100)
n <- 50
p <- 400
X <- matrix(rnorm(n*p), n, p)
b <- rep(0, 400)
b[301:305] <- c(5:1)*2
y <-  X%*%b + rnorm(n)
ynew <-  X%*%b + rnorm(n)    

In [5]:
library(glmnet) 
system.time( fit <- glmnet(X, y, intercept=F, standardize=F, lambda=1) )

Loading required package: Matrix

Warning message:
“package ‘Matrix’ was built under R version 4.0.5”
Loaded glmnet 4.1-3



   user  system elapsed 
  0.007   0.001   0.007 

In [39]:
system.time( re <- admmLasso(X, y,  1.0 * n, tol = 1e-8) ) 

   user  system elapsed 
  0.250   0.008   0.257 

Faster than the CD algorithm

## Note

The objective function solved in ``admmLasso`` is the glmnet objective function ``lasso_obj`` times ``length(y)``. Therefore, ``tau`` should be multiplied by ``n`` in the above to solve the same objective function.

## Comparison

In [40]:
beta_glmnet <- as.numeric(coef(fit)[-1] )

In [41]:
max( abs( beta_glmnet - re )  ) 

[1] 0.001979385

In [42]:
lasso_obj(X, y, beta_glmnet, 1.0)

[1] 26.01695

In [43]:
lasso_obj(X, y, re, 1.0)

[1] 26.01695

In [58]:
lasso_obj(X, y, re, 1.0)  - lasso_obj(X, y, beta_glmnet, 1.0)

[1] -2.646112e-06

# RcppArmadillo

The CPP source file is available on Canvas.

In [45]:
library(Rcpp)
library(RcppArmadillo)

In [48]:
sourceCpp("admm_lasso.cpp")

In [65]:
system.time( beta_cpp <- admm_lasso(X, y, 1.0 * n, 1000, 1e-5) )

   user  system elapsed 
  0.085   0.000   0.085 

In [66]:
max( abs( beta_glmnet - beta_cpp )  ) 

[1] 0.002000808

In [62]:
lasso_obj(X, y, beta_cpp, 1.0)

[1] 26.01695

In [63]:
lasso_obj(X, y,  beta_cpp, 1.0)  - lasso_obj(X, y, beta_glmnet, 1.0)

[1] -2.494906e-06

**Note**: the compiler is not optimized using clang. If using GCC for compiling in **docker**, the *admm_lasso* CPP implementation is actually faster than *glmnet*. See our [slides](https://lovedatascience.com/statcomp/cpp.html).